# 10. 얼굴인식(Haar Cascade)

In [16]:
import cv2 as cv
SONNY = "../images/son.png"
FAMILY = "../images/family.jpg"

FACE_CASCADE = "../cascade/haarcascade_frontalface_default.xml"
EYE_CASCADE = "../cascade/haarcascade_eye.xml"

## 얼굴인식
- `cv2.detectMultiScale(image, scaleFactor, minNeighbors, flags, 
minSize, maxSize)`
    - OpenCV의 객체 검출 함수로, 주로 얼굴 검출과 같은 작업에 사용
    - `scaleFactor` : 이미지 크기를 얼마나 줄여가며 검출을 진행할지를 결정
    - `minNeighbors` 얼마나 많은 주변에 객체로 간주할지를 결정, 값이 크면 더 강한 검출 기준을 설정
    - `minSize` : 검출할 객체의 최소 크기

In [32]:
face_cascade = cv.CascadeClassifier(FACE_CASCADE)
eye_cascade = cv.CascadeClassifier(EYE_CASCADE)

img = cv.imread(SONNY)
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

faces = face_cascade.detectMultiScale(
    gray, scaleFactor=1.1, minNeighbors=2, minSize=(10,10)
)

eyes = eye_cascade.detectMultiScale(
    gray, scaleFactor=1.1, minNeighbors=1, maxSize=(10,10)
)
# print(eyes)
# print(faces)
if len(faces):
    for face in faces:
        x, y, width, height = face
        cv.rectangle(img, (x,y), (x+width, y+height), (0,255,0), 2, cv.LINE_AA)

if len(eyes):
    for eye in eyes:
        x, y, width, height = eye
        cv.rectangle(img, (x,y), (x+width, y+height), (0,255,255), 2, cv.LINE_AA)

cv.imshow("img", img)
cv.waitKey(0)
cv.destroyAllWindows()
